In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
os.chdir("/project/arpae/mtezcan/projects/RAPiD-T/")

import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np 
import glob
from api import Detector
import json
import time

from utils import visualization
from utils import vid_utils as vid

In [2]:
weights_path = "weights/RAPiD_FA.ckpt"
in_path = "examples"
out_path = "outputs/RAPiD_FA"
os.makedirs(out_path, exist_ok=True)
flow_warping = False
detector = Detector(model_name='rapid', weights_path=weights_path)

Using backbone Darknet-53. Loading ImageNet weights....
Number of parameters in backbone: 40584928
Successfully initialized model rapid. Total number of trainable parameters: 72674070
Successfully loaded weights: weights/RAPiD_FA.ckpt
Using CUDA...


In [3]:
vid_names = ['warehouse_short']

In [4]:
vid_conf = 0.3
os.makedirs(os.path.join(out_path, "txts"), exist_ok=True)
os.makedirs(os.path.join(out_path, "jsons"), exist_ok=True)
os.makedirs(os.path.join(out_path, f"vids/conf{vid_conf}"), exist_ok=True)

for vid_name in vid_names:
    print(f"Dealing with {vid_name}")
    vid_folder = os.path.join(in_path, vid_name)
    txt_det_folder = os.path.join(out_path, "txts", vid_name)
    json_det_path = os.path.join(out_path, "jsons", f"{vid_name}_dets.json")
    
    # compute detections
    print("Computing detections...")
    
    tic = time.perf_counter()
    vid.saveDets(
        detector, 
        vid_folder, 
        txt_det_folder, 
        flow_warping=flow_warping,
    )
    toc = time.perf_counter()
    elapsed = toc - tic
    num_frames = len(os.listdir(vid_folder))
    fps = num_frames / elapsed
    print(f"Computed detections of {num_frames} frames in {elapsed:0.4f} seconds: {fps:0.4f} FPS")

    # Conver detections from txt to json
    vid.txt2json(txt_det_folder, json_det_path)
    
    # Produce a video of detections
    print("Creating video...")
    im2dets = vid.json2dict(json_det_path)
    im2dets_arr = [(im2dets, (0, 255, 0), False, vid_conf)]
    vid_path = os.path.join(out_path, f"vids/conf{vid_conf}", f"{vid_name}_dets.mp4")
    vid.saveVideo(vid_folder, im2dets_arr, vid_path)
print("DONE!")

Dealing with warehouse
Computing detections...
Confidence Threshold is 0.05


/share/pkg.7/pytorch/1.9.0/install/lib/python3.8/site-packages/torch/cuda/memory.py:260: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


Computed detections of 49 frames in 22.7470 seconds: 2.1541 FPS
Creating video...
DONE!
